In [1]:
import sys
sys.path.insert(0, 'd:\\Data Science\\projects\\smart photo sharing\\backend')

from src.db.base import SessionLocal
from src.models.face import Face
from src.models.photo import Photo
from src.tasks.workers.search_worker import build_faiss_index_task
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Imports successful")

>>> SECURITY MODULE LOADED (src/core/security.py) <<<
✅ Imports successful


## Check Database for Embeddings

In [2]:
db = SessionLocal()

# Count faces with embeddings
face_count = db.query(Face).filter(Face.embedding.isnot(None)).count()
total_faces = db.query(Face).count()

print(f"📊 Database Stats:")
print(f"  - Total faces: {total_faces}")
print(f"  - Faces with embeddings: {face_count}")

if face_count > 0:
    print(f"\n✅ Found {face_count} faces with embeddings - ready to index")
else:
    print(f"\n⚠️ No faces with embeddings found - need to process photos first")

db.close()

2025-12-26 09:58:11,878 INFO sqlalchemy.engine.Engine select pg_catalog.version()


INFO:sqlalchemy.engine.Engine:select pg_catalog.version()


2025-12-26 09:58:11,880 INFO sqlalchemy.engine.Engine [raw sql] {}


INFO:sqlalchemy.engine.Engine:[raw sql] {}


2025-12-26 09:58:11,896 INFO sqlalchemy.engine.Engine select current_schema()


...

INFO:sqlalchemy.engine.Engine:SELECT count(*) AS count_1 
FROM (SELECT faces.id AS faces_id, faces.photo_id AS faces_photo_id, faces.bbox AS faces_bbox, faces.embedding AS faces_embedding, faces.confidence AS faces_confidence, faces.thumbnail_s3_key AS faces_thumbnail_s3_key, faces.blur_score AS faces_blur_score, faces.brightness_score AS faces_brightness_score, faces.deleted_at AS faces_deleted_at, faces.extra_data AS faces_extra_data, faces.created_at AS faces_created_at, faces.updated_at AS faces_updated_at 
FROM faces) AS anon_1


2025-12-26 09:58:12,036 INFO sqlalchemy.engine.Engine [generated in 0.00557s] {}


INFO:sqlalchemy.engine.Engine:[generated in 0.00557s] {}


📊 Database Stats:
  - Total faces: 10
  - Faces with embeddings: 10

✅ Found 10 faces with embeddings - ready to index
2025-12-26 09:58:12,049 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


## Enqueue Build Task

In [17]:
# Enqueue the global index build task
task = build_faiss_index_task.delay(persist=True, to_s3=False)

print(f"📋 Task queued:")
print(f"  - Task ID: {task.id}")
print(f"  - Status: {task.status}")
print(f"\n🔗 You can check status with: task.state, task.info")

📋 Task queued:
  - Task ID: dcf2c734-5953-4e7d-a0b0-d20b04f22a0b
  - Status: PENDING

🔗 You can check status with: task.state, task.info


## Monitor Task Progress

In [18]:
import time

# Wait for task to complete (with timeout)
timeout = 120  # seconds
start = time.time()

while True:
    elapsed = time.time() - start
    if elapsed > timeout:
        print(f"⏱️ Timeout after {timeout}s")
        break
    
    state = task.state
    info = task.info
    
    print(f"[{elapsed:.0f}s] Status: {state}")
    if isinstance(info, dict):
        print(f"       Info: {info}")
    
    if state in ['SUCCESS', 'FAILURE']:
        print(f"\n✅ Task completed: {state}")
        if state == 'SUCCESS':
            print(f"Result: {task.result}")
        break
    
    time.sleep(5)

[0s] Status: SUCCESS
       Info: {'status': 'completed', 'faces_indexed': 10, 'album_id': None, 'persist_queued': True}

✅ Task completed: SUCCESS
Result: {'status': 'completed', 'faces_indexed': 10, 'album_id': None, 'persist_queued': True}


In [14]:
# Get detailed error info
if task.state == 'FAILURE':
    print(f"❌ Task failed with error:")
    print(f"Exception: {task.info}")
    if hasattr(task, 'traceback'):
        print(f"\nTraceback:\n{task.traceback}")

❌ Task failed with error:
Exception: Never call result.get() within a task!
See https://docs.celeryq.dev/en/latest/userguide/tasks.html#avoid-launching-synchronous-subtasks


Traceback:
Traceback (most recent call last):
  File "C:\Users\tejas\AppData\Local\pypoetry\Cache\virtualenvs\backend-esi-9Eed-py3.10\lib\site-packages\celery\app\trace.py", line 453, in trace_task
    R = retval = fun(*args, **kwargs)
  File "C:\Users\tejas\AppData\Local\pypoetry\Cache\virtualenvs\backend-esi-9Eed-py3.10\lib\site-packages\celery\app\trace.py", line 736, in __protected_call__
    return self.run(*args, **kwargs)
  File "D:\Data Science\projects\smart photo sharing\backend\src\tasks\workers\search_worker.py", line 323, in build_faiss_index_task
    persist_index_task.apply_async(
  File "C:\Users\tejas\AppData\Local\pypoetry\Cache\virtualenvs\backend-esi-9Eed-py3.10\lib\site-packages\celery\result.py", line 237, in get
    assert_will_not_block()
  File "C:\Users\tejas\AppData\Local\pypoetry\Cache\

## Verify Index

In [19]:
import os

index_path = os.path.join("data", "models", "faiss", "index")

# Check if index files exist
faiss_exists = os.path.exists(f"{index_path}.faiss")
pkl_exists = os.path.exists(f"{index_path}.pkl")

print(f"📁 Index Files:")
print(f"  - {index_path}.faiss: {'✅ exists' if faiss_exists else '❌ missing'}")
print(f"  - {index_path}.pkl: {'✅ exists' if pkl_exists else '❌ missing'}")

if faiss_exists and pkl_exists:
    faiss_size = os.path.getsize(f"{index_path}.faiss") / 1024 / 1024  # MB
    pkl_size = os.path.getsize(f"{index_path}.pkl") / 1024  # KB
    print(f"\n📊 Index Size:")
    print(f"  - FAISS: {faiss_size:.2f} MB")
    print(f"  - Mappings: {pkl_size:.2f} KB")

📁 Index Files:
  - data\models\faiss\index.faiss: ✅ exists
  - data\models\faiss\index.pkl: ✅ exists

📊 Index Size:
  - FAISS: 0.02 MB
  - Mappings: 0.42 KB


## Load and Test Index

In [20]:
from src.tasks.workers.search_worker import GLOBAL_FAISS_ENGINE, init_global_faiss
import numpy as np

# Initialize and load global engine
faiss_engine = init_global_faiss()
faiss_engine.load(index_path)

# Get index stats
ntotal = faiss_engine.index.ntotal
is_trained = faiss_engine.index.is_trained

print(f"🔍 FAISS Index Stats:")
print(f"  - Total vectors: {ntotal}")
print(f"  - Is trained: {is_trained}")
print(f"  - ID mappings: {len(faiss_engine.id_map)}")

if ntotal > 0:
    print(f"\n✅ Index loaded successfully with {ntotal} faces")
else:
    print(f"\n⚠️ Index is empty")

INFO:src.tasks.workers.search_worker:🔧 Initializing global FAISS engine...
INFO:src.services.face.pipeline:🔧 Initializing FAISS on CPU


🔍 FAISS Index Stats:
  - Total vectors: 10
  - Is trained: True
  - ID mappings: 10

✅ Index loaded successfully with 10 faces


## Test Search (if index has faces)

In [21]:
if ntotal > 0:
    # Create random query embedding
    query = np.random.randn(1, 512).astype(np.float32)
    query /= np.linalg.norm(query)
    
    # Search
    scores, indices = faiss_engine.index.search(query, k=5)
    
    print(f"🔎 Search Results (top 5):")
    for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
        if idx == -1:
            break
        face_id = faiss_engine.id_map.get(idx, 'unknown')
        print(f"  {i+1}. Face ID: {face_id}, Similarity: {score:.4f}")
else:
    print("⚠️ Cannot test search - index is empty")

🔎 Search Results (top 5):
  1. Face ID: 311eff98-b906-435d-bb42-16b54c5f5a76, Similarity: 0.1234
  2. Face ID: 62e75e76-cd6b-4113-968e-40b37a82a592, Similarity: 0.1234
  3. Face ID: ae7626b1-8337-472e-b086-cb76ab8e23e1, Similarity: 0.0999
  4. Face ID: 29f79148-fb2f-4535-9df5-2b2963eba965, Similarity: 0.0900
  5. Face ID: 100112d0-2ebe-4fdd-b9f2-d78779e9cb1c, Similarity: 0.0900


## Summary

In [22]:
print(f"""\n{'='*60}
📊 FAISS Index Status Summary
{'='*60}
Indexed Faces: {ntotal}
Index Trained: {is_trained}
Index Path: {index_path}
Files Saved: {faiss_exists and pkl_exists}
Ready for Search: {ntotal > 0 and is_trained}
{'='*60}
""")


📊 FAISS Index Status Summary
Indexed Faces: 10
Index Trained: True
Index Path: data\models\faiss\index
Files Saved: True
Ready for Search: True

